In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [ ]:
df = pd.read_csv("/kaggle/input/brasilian-houses-to-rent/houses_to_rent.csv")
df.head()

In [ ]:
print(f"Rows:{df.shape[0]}")
print(f"Colunms:{df.shape[1]}")

In [ ]:
df = df.drop(df.columns[0],axis=1)

In [ ]:
def missing_data(data):
    missing_data = df.isnull().sum() #total de null    
    total = missing_data.sort_values(ascending=True) #ordenação 
    percent = (missing_data / len(df.index)*100).round(2).sort_values(ascending=True) #cálculo percentual
    missing_table = pd.concat([total, percent], axis=1, keys=['Qty NaN', '% NaN'])
    return missing_table

In [ ]:
missing_data(df)

In [ ]:
df_ = df.loc[df['city'] == 1]
df_.head()

In [ ]:
df_['floor'] = df_['floor'].replace('-',0)

for col in ['hoa', 'rent amount', 'property tax', 'fire insurance', 'total']:
    df_[col] = df_[col].replace('R\$','',regex=True)
    df_[col] = df_[col].replace(',','',regex=True)

df_['hoa'] = df_['hoa'].replace('Sem info', value='0')
df_['hoa'] = df_['hoa'].replace('Incluso', value='0')
df_['property tax'] = df_['property tax'].replace('Incluso', value='0')

def categorical(df, col):
    df[col] = pd.Categorical(df[col])
    return df[col].cat.codes

df_['animal'] = categorical(df_, 'animal')
df_['furniture'] = categorical(df_,'furniture')

In [ ]:
df_.head()

In [ ]:
df_.dtypes

In [ ]:
df_.astype(np.int64)

In [ ]:
y = df_['rent amount']
X = df_.drop(['rent amount','total'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=0)

In [ ]:
linear_model = LinearRegression()

linear_model.fit(X_train,y_train)

print("Linear Score: " + str(linear_model.score(X_train,y_train)))

In [ ]:
linear_pred = linear_model.predict(X_test)

print("Linear R2_Score: " + str(r2_score(linear_pred,y_test)))


In [ ]:
predict = pd.DataFrame()

predict['Real'] = y_test
predict['Forecast'] = linear_pred.round()

predict.head(15)